# Support Intelligence & Risk Monitoring — Data Processing Notebook (T3)

**Goal:** Build a clean, ML-ready and monitoring-ready dataset from raw support tickets (EN subset).  
**Output:** `data/processed/tickets_clean_en.parquet`

---

## Why this step matters
From T2 EDA we observed:
- **Missing subjects (~16%)** → we must build a robust `message` field.
- **Tags are optional** (`tag_1..tag_8` contain NaNs) → we normalize them into a list and a string.
- We need a **single analytics-ready source** for:
  - modeling (category triage, priority prediction)
  - monitoring (spikes by queue/tag/priority)
  - future ETL/DB/API steps

---

## What we build in this notebook
1. Load raw dataset and filter **English tickets**
2. Clean text fields (`subject_clean`, `body_clean`)
3. Handle missing subjects (`subject_filled`, `has_subject`)
4. Create `message` (subject + body fallback)
5. Normalize tags (`tags`, `tags_str`, `n_tags`)
6. Add lightweight features (`body_len`, `message_len`, `is_very_short`)
7. (V1) Map to business categories (**Billing / Bug / Account / Other**)
8. Export `tickets_clean_en.parquet`


In [1]:
import pandas as pd
import numpy as np
import re
import os

from IPython.display import display


## 0) Paths + Load raw dataset

In [2]:
PATH = "C:/Users/hp/Desktop/mon portfolio/Support intellegence and risk monitoring system/archive(1)/aa_dataset-tickets-multi-lang-5-2-50-version.csv"
OUT_DIR = "data/processed"
OUT_PATH = os.path.join(OUT_DIR, "tickets_clean_en.parquet")

df = pd.read_csv(PATH)
print("Raw shape:", df.shape)
df.head(3)


Raw shape: (28587, 16)


,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN


## 1) Filter English (EN)

We start with EN only to keep a clean baseline. Multi-language can be added later.


In [3]:
df_en = df[df["language"] == "en"].copy()
print("EN shape:", df_en.shape)
df_en[["language"]].value_counts().head()


EN shape: (16338, 16)


language
en          16338
Name: count, dtype: int64

## 2) Text cleaning utilities

We apply a *light* cleaning strategy:
- normalize whitespace
- remove newlines/tabs
- keep content readable (no aggressive stemming)


In [4]:
def clean_text(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s)
    s = s.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s


## 3) Clean subject/body + handle missing subjects

In [5]:
df_en["subject_clean"] = df_en["subject"].apply(clean_text)
df_en["body_clean"] = df_en["body"].apply(clean_text)

df_en["has_subject"] = (df_en["subject_clean"].str.len() > 0).astype(int)
df_en["subject_filled"] = df_en["subject_clean"].where(df_en["has_subject"] == 1, "No subject")

print("Subject missing after cleaning:", (df_en["has_subject"]==0).sum(), "/", len(df_en))
df_en[["subject", "subject_clean", "has_subject", "subject_filled"]].head(5)


Subject missing after cleaning: 2607 / 16338


,subject,subject_clean,has_subject,subject_filled
1,Account Disruption,Account Disruption,1,Account Disruption
2,Query About Smart Home System Integration Feat...,Query About Smart Home System Integration Feat...,1,Query About Smart Home System Integration Feat...
3,Inquiry Regarding Invoice Details,Inquiry Regarding Invoice Details,1,Inquiry Regarding Invoice Details
4,Question About Marketing Agency Software Compa...,Question About Marketing Agency Software Compa...,1,Question About Marketing Agency Software Compa...
5,Feature Query,Feature Query,1,Feature Query


## 4) Build `message` (robust text field)

Rule:
- If subject exists → `subject_clean + " | " + body_clean`
- Else → `body_clean` only


In [6]:
df_en["message"] = np.where(
    df_en["has_subject"] == 1,
    df_en["subject_clean"] + " | " + df_en["body_clean"],
    df_en["body_clean"]
)

df_en[["subject_filled", "body_clean", "message"]].head(3)


,subject_filled,body_clean,message
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...",Account Disruption | Dear Customer Support Tea...
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Query About Smart Home System Integration Feat...
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",Inquiry Regarding Invoice Details | Dear Custo...


## 5) Normalize tags (`tag_1..tag_8` → `tags`, `tags_str`, `n_tags`)

Tags are optional. We:
- remove NaN / empty / 'none' tokens
- keep unique tags while preserving order


In [7]:
tag_cols = [c for c in df_en.columns if re.fullmatch(r"tag_\d+", str(c))]
print("Tag columns:", tag_cols)

def collect_tags(row) -> list:
    tags = []
    for c in tag_cols:
        v = row.get(c)
        if pd.isna(v):
            continue
        v = str(v).strip()
        if v == "" or v.lower() in {"nan", "none", "null"}:
            continue
        tags.append(v)

    seen = set()
    out = []
    for t in tags:
        if t not in seen:
            out.append(t)
            seen.add(t)
    return out

df_en["tags"] = df_en.apply(collect_tags, axis=1)
df_en["n_tags"] = df_en["tags"].apply(len)
df_en["tags_str"] = df_en["tags"].apply(lambda xs: " | ".join(xs))

df_en[["tag_1","tag_2","tag_3","tags","n_tags","tags_str"]].head(5)


Tag columns: ['tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8']


,tag_1,tag_2,tag_3,tags,n_tags,tags_str
1,Account,Disruption,Outage,"[Account, Disruption, Outage, IT, Tech Support]",5,Account | Disruption | Outage | IT | Tech Support
2,Product,Feature,Tech Support,"[Product, Feature, Tech Support]",3,Product | Feature | Tech Support
3,Billing,Payment,Account,"[Billing, Payment, Account, Documentation, Fee...",5,Billing | Payment | Account | Documentation | ...
4,Product,Feature,Feedback,"[Product, Feature, Feedback, Tech Support]",4,Product | Feature | Feedback | Tech Support
5,Feature,Product,Documentation,"[Feature, Product, Documentation, Feedback]",4,Feature | Product | Documentation | Feedback


## 6) Add lightweight features (ops-friendly)

These features are useful for:
- modeling (baseline)
- anomaly monitoring (e.g., sudden rise in short/empty messages)


In [8]:
df_en["body_len"] = df_en["body_clean"].str.len()
df_en["message_len"] = df_en["message"].str.len()
df_en["is_very_short"] = (df_en["body_len"] < 30).astype(int)

df_en[["body_len","message_len","is_very_short"]].describe()


,body_len,message_len,is_very_short
count,16338.000000,16338.000000,16338.000000
mean,367.881136,407.053250,0.004958
std,179.890305,179.654487,0.070239
min,6.000000,18.000000,0.000000
25%,214.000000,256.000000,0.000000
50%,377.000000,418.000000,0.000000
75%,533.000000,567.000000,0.000000
max,1147.000000,1191.000000,1.000000


## 7) Normalize priority values

Safety step: normalize and keep only expected labels.


In [9]:
df_en["priority_norm"] = df_en["priority"].astype(str).str.lower().str.strip()
valid_priorities = {"low","medium","high"}
before = len(df_en)
df_en = df_en[df_en["priority_norm"].isin(valid_priorities)].copy()
after = len(df_en)

print("Filtered invalid priorities:", before - after)
df_en["priority_norm"].value_counts()


Filtered invalid priorities: 0


priority_norm
medium    6618
high      6346
low       3374
Name: count, dtype: int64

## 8) V1 Business taxonomy mapping (Billing / Bug / Account / Other)

This is a **rule-based baseline** (interpretable + fast).  
Later, you can replace it with a supervised model.


In [10]:
def map_category(row) -> str:
    q = str(row["queue"]).lower()
    msg = str(row["message"]).lower()
    tags = [t.lower() for t in row["tags"]]

    if any(k in q for k in ["billing", "payment", "refund", "invoice"]) or any(k in msg for k in ["refund", "charge", "invoice", "payment"]):
        return "Billing"

    if any(k in msg for k in ["error", "bug", "crash", "exception", "stack trace", "500", "503"]) or any("bug" in t for t in tags):
        return "Bug"

    if any(k in msg for k in ["login", "password", "account", "locked", "2fa", "verification", "reset"]) or any(k in q for k in ["account", "security"]):
        return "Account"

    return "Other"

df_en["category_mapped"] = df_en.apply(map_category, axis=1)
df_en["category_mapped"].value_counts()


category_mapped
Other      9736
Bug        4320
Billing    1603
Account     679
Name: count, dtype: int64

## 9) Final selection of columns (clean dataset)

We keep raw fields for traceability and add clean fields + features.


In [11]:
keep = [
    "subject","body","answer","type","queue","priority","language","version",
    "subject_clean","body_clean","has_subject","subject_filled","message",
    "tags","tags_str","n_tags",
    "body_len","message_len","is_very_short",
    "priority_norm",
    "category_mapped"
]

df_clean = df_en[keep].copy()
print("Final clean shape:", df_clean.shape)
df_clean.head(3)


Final clean shape: (16338, 21)


,subject,body,answer,type,queue,priority,language,version,subject_clean,body_clean,...,subject_filled,message,tags,tags_str,n_tags,body_len,message_len,is_very_short,priority_norm,category_mapped
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...",...,Account Disruption,Account Disruption | Dear Customer Support Tea...,"[Account, Disruption, Outage, IT, Tech Support]",Account | Disruption | Outage | IT | Tech Support,5,544,565,0,high,Account
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",...,Query About Smart Home System Integration Feat...,Query About Smart Home System Integration Feat...,"[Product, Feature, Tech Support]",Product | Feature | Tech Support,3,534,587,0,medium,Other
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",...,Inquiry Regarding Invoice Details,Inquiry Regarding Invoice Details | Dear Custo...,"[Billing, Payment, Account, Documentation, Fee...",Billing | Payment | Account | Documentation | ...,5,605,641,0,low,Billing


## 10) Quick quality report (post-processing)

Sanity checks after transformation.


In [12]:
print("Missing % (key cols):")
display((df_clean[["message","priority_norm","queue","type"]].isna().mean()*100).round(3))

print("\nSubject missing handled (has_subject=0):", int((df_clean["has_subject"]==0).sum()))
print("No-tag tickets:", int((df_clean["n_tags"]==0).sum()))
print("\nCategory distribution:")
display(df_clean["category_mapped"].value_counts())


Missing % (key cols):


message          0.0
priority_norm    0.0
queue            0.0
type             0.0
dtype: float64


Subject missing handled (has_subject=0): 2607
No-tag tickets: 0

Category distribution:


category_mapped
Other      9736
Bug        4320
Billing    1603
Account     679
Name: count, dtype: int64

## 11) Export (Parquet)

Parquet is fast and compact, ideal for:
- modeling notebooks
- later ETL into a database
- API/dashboards pipelines


In [17]:
OUT_DIR = "data/processed"
os.makedirs(OUT_DIR, exist_ok=True)
csv_path = os.path.join(OUT_DIR, "tickets_clean_en.csv")
df_clean.to_csv(csv_path, index=False, encoding="utf-8")
print("Saved CSV:", csv_path)


Saved CSV: data/processed\tickets_clean_en.csv


## 12) Next steps (T4 / Modeling)

After this notebook, you can:
- Build a **tag dictionary** (definitions + examples per tag)
- Train baselines:
  - Ticket category classifier (Billing/Bug/Account/Other)
  - Priority prediction model (high/medium/low)
- Prepare monitoring metrics:
  - tickets per queue/day
  - high-priority rate by queue
  - spikes in specific tags
